# Solve EEG forward problem

In this example, we go through the process of solving EEG forward problem for the model generated in [this example](1_model_creation.ipynb). The result of this process is a `EEGLeadfieldMatrix` which can be used for further computations.

## Model loading

First we load the previously generated `FEModel`.

In [1]:
from pathlib import Path
from shamo.model import FEModel

model_name = "example_model"
model_path = Path(
    "./derivatives/{name}/{name}.json".format(name=model_name))
model = FEModel.load(model_path)

Now the model is loaded and we can use it to compute a leadfield matrix.

## Problem definition

In order to generate a `EEGLeadfieldMatrix`, we must define an `EEGForwardProblem`.

In [2]:
from shamo.problem.forward.eeg import EEGForwardProblem

problem = EEGForwardProblem()

We can now specify the settings of the problem. As we are trying to solve the EEG forward problem, we must set the electrical conductivity of all the tissues present in the model. Two methods can be used:
- `add_tissue()` to add tissues one-by-one.
- `add_tissues()` to add multiple tissues.

> **Note:**  
> Here we set σ [S/m] to some arbitrary values but you should always make sure to use the proper values for the tissues you define.

In [3]:
problem.add_tissue("a", 1.0)
tissues = {"b": 0.5, "c": 0.25}
problem.add_tissues(tissues, True)

{'problem_type': 'eeg',
 'tissue_properties': {'a': {'sigma': 1.0, 'use_anisotropy': False},
  'b': {'sigma': 0.5, 'use_anisotropy': True},
  'c': {'sigma': 0.25, 'use_anisotropy': True}},
 'undersampling_distance': 0,
 'markers': []}

Now that the tissues are all setup, we can focus on the sensors. The problem requires a reference which is the ground sensor. We add it by its name in the model.

In [4]:
problem.set_reference("A")

{'problem_type': 'eeg',
 'tissue_properties': {'a': {'sigma': 1.0, 'use_anisotropy': False},
  'b': {'sigma': 0.5, 'use_anisotropy': True},
  'c': {'sigma': 0.25, 'use_anisotropy': True}},
 'undersampling_distance': 0,
 'markers': [],
 'reference': 'A'}

We can also list the names of the sensors which should not be considered in the computation. Those are called "markers". We can add them using both:
- `add_marker()` to add only one sensor to the list.
- `add_markers()` to add several sensors at once.

For the sake of this example, we will add two markers.

In [5]:
problem.add_markers(["C", "G"])

{'problem_type': 'eeg',
 'tissue_properties': {'a': {'sigma': 1.0, 'use_anisotropy': False},
  'b': {'sigma': 0.5, 'use_anisotropy': True},
  'c': {'sigma': 0.25, 'use_anisotropy': True}},
 'undersampling_distance': 0,
 'markers': ['C', 'G'],
 'reference': 'A'}

Finaly, we must list the tissues for which we want to compute the leadfield matrix. Once again, it is possible to add them either one by one or by batchs using the following methods:
- `add_region_of_interest()`
- `add_regions_of_interest()`

In [6]:
problem.add_regions_of_interest(["b", "c"])

{'problem_type': 'eeg',
 'tissue_properties': {'a': {'sigma': 1.0, 'use_anisotropy': False},
  'b': {'sigma': 0.5, 'use_anisotropy': True},
  'c': {'sigma': 0.25, 'use_anisotropy': True}},
 'undersampling_distance': 0,
 'markers': ['C', 'G'],
 'reference': 'A',
 'region_of_interest': ['b', 'c']}

It is possible to undersample the resulting matrix to only keep values for elements equally spaced by a certain distance [mm].

> **Note:**  
> This parameter will induce some loss of data. You should always make sure to have enough data for the purpose you are doing these computations for.

In [7]:
problem.set_undersampling_distance(50)

{'problem_type': 'eeg',
 'tissue_properties': {'a': {'sigma': 1.0, 'use_anisotropy': False},
  'b': {'sigma': 0.5, 'use_anisotropy': True},
  'c': {'sigma': 0.25, 'use_anisotropy': True}},
 'undersampling_distance': 50,
 'markers': ['C', 'G'],
 'reference': 'A',
 'region_of_interest': ['b', 'c']}

## Problem resolution

Now that the problem is completely defined, we just have to solve it.

In [8]:
leadfield = problem.solve(model, 
                          "example_eeg_leadfield", 
                          "./derivatives")